In [1]:
import numpy as np
import pandas as pd
import pymongo
import bson

In [73]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Froce Brute search"""
    def searchInDBForceBrute(self, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = []
            
            self.searchInCollection(collection, )
            print(self.getItems(collection))
            for element in self.getItems(collection)[0]:
                for key in element.keys():
                    query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
                    
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
                
        
MS = MongoServer(True)   
print(MS.findNElement('ratings', 10))
"""
recipes = MS.getCollection('recipes')
print(recipes.find_one())

ratings = MS.getCollection('ratings')
print(ratings.find_one())

ratings = MS.getCollection('ingredients')
print(ratings.find_one())

users = MS.getCollection('users')
print(users.find_one())


print(MS.searchInDB('_id', users.find_one()['_id']))

#########################################33

MS.getAllCollections()
for col in MS.collections:
    print(col, "\n")

#############################################33

print("users_recipes_ratings: ")
print("\t ", MS.findOne('users_recipes_ratings'), "\n")

print("ratings: ")
print("\t ", MS.findOne('ratings'), "\n")

print("recipes: ")
print("\t ", MS.findOne('recipes'), "\n")

print("users: ")
print("\t ", MS.findOne('users'), "\n")

print("recipes_ingredients: ")
print("\t ", MS.findOne('recipes_ingredients'), "\n")

print("ingredients: ")
print("\t ", MS.findOne('ingredients'), "\n")
"""

[{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4'), 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'), 'rating': 1}, {'_id': ObjectId('5a1aa9612cfaa80d6ff25316'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4'), 'recipe_id': ObjectId('5a073c312cfaa84d14994994'), 'rating': 1}, {'_id': ObjectId('5a1aa9632cfaa80d6ff2531f'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f7'), 'recipe_id': ObjectId('5a073c312cfaa84d14995dc5'), 'rating': 3}, {'_id': ObjectId('5a1aa9632cfaa80d6ff25320'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d14996b2f'), 'rating': 1}, {'_id': ObjectId('5a1aa9632cfaa80d6ff25321'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d14994986'), 'rating': 1}, {'_id': ObjectId('5a1aa9632cfaa80d6ff25322'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d1499541b'), 'rating': 4}, {'_id': ObjectId('5a1aa9602cfaa80d6ff25

'\nrecipes = MS.getCollection(\'recipes\')\nprint(recipes.find_one())\n\nratings = MS.getCollection(\'ratings\')\nprint(ratings.find_one())\n\nratings = MS.getCollection(\'ingredients\')\nprint(ratings.find_one())\n\nusers = MS.getCollection(\'users\')\nprint(users.find_one())\n\n\nprint(MS.searchInDB(\'_id\', users.find_one()[\'_id\']))\n\n#########################################33\n\nMS.getAllCollections()\nfor col in MS.collections:\n    print(col, "\n")\n\n#############################################33\n\nprint("users_recipes_ratings: ")\nprint("\t ", MS.findOne(\'users_recipes_ratings\'), "\n")\n\nprint("ratings: ")\nprint("\t ", MS.findOne(\'ratings\'), "\n")\n\nprint("recipes: ")\nprint("\t ", MS.findOne(\'recipes\'), "\n")\n\nprint("users: ")\nprint("\t ", MS.findOne(\'users\'), "\n")\n\nprint("recipes_ingredients: ")\nprint("\t ", MS.findOne(\'recipes_ingredients\'), "\n")\n\nprint("ingredients: ")\nprint("\t ", MS.findOne(\'ingredients\'), "\n")\n'

In [74]:
recipes = MS.getItems('ratings')

Collection  ratings  Update in Local.


In [51]:
recipes = MS.getItems('recipes_ingredients')
len(recipes)

2467

In [45]:
recipes[-1]

{'_id': ObjectId('5a073c312cfaa84d14996b3f'),
 'id': 99990,
 'image': 'http://www.seriouseats.com/recipes/assets_c/2010/08/20100527-grilled-pizza-opener-thumb-560x373-91806-thumb-625xauto-108910.jpg',
 'list_ingredients': ['1 (14.4 ounce) can whole peeled tomatoes(preferably San Marzano)(drained)',
  'Kosher salt',
  '1/2 cup extra virgin olive oil',
  '1 pound shredded or thinly sliced low-moisture mozzarella cheese (see note above)',
  '4 ounces roughly grated Parmiggiano-Reggiano or Pecorino Romano',
  '1/2 cup finely chopped basil leaves'],
 'name': 'Grilled Pizza Napoletana',
 'ningredients': 6.0,
 'serves': 'Makes 4 pizzas',
 'time_min': '2 hours (plus time for dough to proof)',
 'url': 'http://www.seriouseats.com/recipes/2010/05/how-to-make-grilled-pizza-napoletana-on-a-barbecue-recipe.html'}

In [46]:
query = MS.searchInCollection('recipes', 'id', 236853)[0]

In [47]:
MS.searchInCollection('recipes', '_id', query['_id'])

[{'_id': ObjectId('5a073c312cfaa84d14995243'),
  'id': 236853,
  'image': 'http://www.seriouseats.com/recipes/assets_c/2013/01/20120111-bakethebook-tkocookies-thumb-625xauto-298991.jpg',
  'list_ingredients': ['4 ounces (125 grams) 35% white chocolate(chopped)',
   '0.5 ounce (15 grams) unsalted butter ',
   '1/2 cup + 1 teaspoon  (125 grams) heavy cream ',
   '1 3/4 cups + 1 1/2 tablespoons (259 grams) all-purpose flour ',
   '1 cup + 1 1/2 tablespoons  (87 grams) unsweetened alkalized cocoa powder ',
   '3/8 teaspoon (1.6 grams) baking soda ',
   '8 ounces (227 grams) unsalted butter ',
   '2 teaspoons (6 grams) Kosher salt ',
   '3/4 cup + 1 tablespoon (161 grams) Granulated sugar '],
  'name': "Bouchon Bakery's TKO Cookies",
  'ningredients': 9.0,
  'serves': 'makes 8 3-inch sandwich cookies',
  'time_min': '1 hour',
  'url': 'http://www.seriouseats.com/recipes/2013/01/thomas-keller-bouchon-bakery-tko-chocolate-sandwich-cookie-recipe.html'}]

In [60]:
_dict = MS.findOne('ratings')
_dict

{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'),
 'rating': 1,
 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'),
 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')}

In [50]:
type(1.)

float

In [65]:
query_rates = MS.searchInCollection(name_collection='ratings', field='user_id', patro=_dict['user_id'])
for q in query_rates:
    print(MS.searchInCollection(name_collection='recipes', field='_id', patro=q['recipe_id']))

[{'_id': ObjectId('5a073c312cfaa84d14994d8e'), 'id': 212491, 'name': "Susan Spicer's Pan-Roasted Chicken Breast with Vinegar, Mustard, and Tarragon", 'url': 'http://www.seriouseats.com/recipes/2012/07/susan-spicers-pan-roasted-chicken-breast-with-vinegar-mustard-and-tarragon.html', 'image': 'http://www.seriouseats.com/recipes/assets_c/2012/06/20120627-212491-dinner-tonight-susan-spicer-pan-roasted-chicken-breast-primary-thumb-625xauto-253023.jpg', 'serves': 'serves 4', 'time_min': '30 minutes', 'list_ingredients': ['4 boneless chicken breasts (about 6 ounces each)(with skin still attached)', 'Salt and pepper', '2 tablespoons canola oil or olive oil ', '1 cup chicken stock ', '6 tablespoons apple cider vinegar ', '2 tablespoons butter(softened)', '2 teaspoons Dijon mustard', '2 teaspoons coarsely chopped tarragon leaves', '2 teaspoons chopped fresh chives'], 'ningredients': 9.0}]
[{'_id': ObjectId('5a073c312cfaa84d14994994'), 'id': 19396, 'name': 'Cook the Book: Cowgirl Steaks with Pink

In [73]:
query_rates

[{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'),
  'rating': 1,
  'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'),
  'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')},
 {'_id': ObjectId('5a1aa9612cfaa80d6ff25316'),
  'rating': 1,
  'recipe_id': ObjectId('5a073c312cfaa84d14994994'),
  'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')},
 {'_id': ObjectId('5a1aa9612cfaa80d6ff25317'),
  'rating': 3,
  'recipe_id': ObjectId('59fb0d77eee3642f3c068b92'),
  'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')}]

In [55]:
class Recommender:
    
    def __init__(self):
        # connect to mongo with MongoServer object
        self.server = MongoServer(True)
        
    def dummieRecommendation(self, N = 10):
        listObjectIds = []
        for item in server.findNElement('recipes', N):
            listObjectIds.append(item['_id'])
        return listObjectIds
        
    
        
    def isObjectId(self, _id):
        try:
            # Do a query in a specific user collection
            if not type(_id) == bson.objectid.ObjectId:
                # creation of a objectID
                if type(_id) == str:
                    idUser = bson.objectid.ObjectId(_id)
                else:
                    return None
            return _id
        except:
            return None
            
    """
    Search user by idRecepie(ObjectId)
    """ 
    def searchRecepieWithIngredientsByIs(self, idRecepie):
        idRecepie = self.isObjectId(idRecepie)
        if idRecepie == None:
            print ("idRecepie is not a ObjectId")
            return []
        
        # Search the recepie    
        _collection = 'recipes_ingredients'
        _field = '_id' # ObjectId
        _patro = idRecepie
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]    
        # obtain the ingredients
        return query, query.values()[1]
    

    """
    Search user by idUser(ObjectId)
    """
    def searchUsersById(self, idUser):
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        
        _collection = 'users'
        _field = '_id' # ObjectId
        _patro = idUser
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]
        
        return query
    
    def computeRecommenderMatrixBestRated(self, idRecipe):
        # look the ingredients used in the recipe
        # look the recipe that use on of the ingridients
        # compute Recommender Matrix 
        matrix = pd.DataFrame(random, index=ids, columns=user)
        return matrix
    
    def computeRecommenderMatrixCollaborativeFiltering(self, idUser):
        m_ids = []
        m_user = []
        
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        # look for the ratings of the user
        ratingsUser = self.server.searchInCollection(name_collection='ratings', field='user_id', patro=idUser)
        if ratingsUser == []:
            print ("User has no ratings. Cold Start.")
            return None
        ratings = []
        m_user.append(idUser) 
        # obtain ratings from the same recipes
        for rating in ratingsUser:
            m_ids.append(rating['recipe_id'])
            # search for more ratings in the same recipe
            recipes = self.server.searchInCollection(name_collection='ratings', field='recipe_id', patro=rating['recipe_id'])
            
            # acumulate the ratings
            ratings += recipes
            
            # look if objectId of recipe is in the list
            for recipe in recipes:
                if not recipe['recipe_id'] in m_ids:
                    m_ids.append(recipe['recipe_id'])
        
        print ("Recipes ids: ", m_ids)
        print ("Users ids: ", m_user)
            
        # compute Recommender Matrix 
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
    
    def generateRatingMatrix(self):
        m_ids = set()
        m_user = set()
        ratings = self.server.getItems('ratings')
        for item in ratings:
            m_ids.add(item["recipe_id"])
            m_user.add(item["user_id"])
        
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
            
    def computeRecomendation(self):
        # Introduce here the distance function for each of the cases
        # maybe it is necesary to separete the funciton in two
        pass
    
    def bestRated(self, idRecepie, n):
        # take the recepie
        # look for all the recepie containing similar ingridients
        # generate the recommender matrix recepie - ingridient
        return self.dummieRecommendation(N)
    
    def collaborativeFiltering(self, idUser):
        # take the ratings of the user
        # with the recipes of the user, find which recepes we can generate
        # generate the recommender matrix for user
        # call the distance function
        return self.dummieRecommendation(N)
    
    def searchByTngredients(self, listIngredients, N = 10):
        return self.dummieRecommendation(N)
        
    
    
rec = Recommender()

#rec.computeRecommenderMatrixCollaborativeFiltering(bson.objectid.ObjectId('5a1aa2cb2cfaa80d6ff251f3'))
#rec.generateRatingMatrix()

In [36]:
rec.server.searchInCollection(name_collection='ratings', field='user_id', patro=bson.objectid.ObjectId('5a1aa9602cfaa80d6ff25314'))

[]

In [41]:
rec.server.getItems('users_recipes_ratings')

[{'_id': ObjectId('5a298de1166d3310bc8207d1'),
  'rating': 4,
  'recipe': '236853',
  'user': 'user_1'},
 {'_id': ObjectId('5a298de1166d3310bc8207d2'),
  'rating': 5,
  'recipe': '241469',
  'user': 'user_1'},
 {'_id': ObjectId('5a298de1166d3310bc8207d3'),
  'rating': 5,
  'recipe': '85980',
  'user': 'user_1'},
 {'_id': ObjectId('5a298de1166d3310bc8207d4'),
  'rating': 5,
  'recipe': '224046',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207d5'),
  'rating': 5,
  'recipe': '297457',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207d6'),
  'rating': 3,
  'recipe': '299052',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207d7'),
  'rating': 3,
  'recipe': '297612',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207d8'),
  'rating': 4,
  'recipe': '297836',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207d9'),
  'rating': 5,
  'recipe': '224507',
  'user': 'user_2'},
 {'_id': ObjectId('5a298de1166d3310bc8207da'),
  'rating

In [48]:
rec.server.findOne('ratings')

{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'),
 'rating': 1,
 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'),
 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')}

In [46]:
rec.server.finde('recipes_ingredients')

[{'_id': ObjectId('5a29a690166d3310bc82f132'),
  'chicken-pumpkin-curry': ['ghee',
   'oil',
   'onion',
   'pack',
   'asda',
   'chicken',
   'thigh',
   'fillet',
   'skin',
   'garlic',
   'clove',
   'ginger',
   'turmeric',
   'cumin',
   'asda',
   'garam',
   'masala',
   'chillie',
   'can',
   'coconut',
   'milk',
   'pumpkin',
   'skin',
   'fibre',
   'cornflour',
   'cold',
   'water',
   'make',
   'paste',
   'coriander',
   'rice',
   'serve']},
 {'_id': ObjectId('5a29a690166d3310bc82f133'),
  'masterclass-pulled-ham-creamy-herb-salad': ['butcher’',
   'gammon',
   'joint',
   'cider',
   'bag',
   'asda',
   'tender',
   'leaf',
   'salad',
   'caper',
   'frozen',
   'pea',
   'minute',
   'radishe',
   'chosen',
   'gherkin',
   'cider',
   'vinegar',
   'olive',
   'oil',
   'fat',
   'crème',
   'fraîche',
   'dijon',
   'mustard',
   'parsley']},
 {'_id': ObjectId('5a29a690166d3310bc82f134'),
  'banana-pecan-date-muffins': ['butter',
   'plu',
   'light',
   'sof

In [47]:
rec.server.findOne('recipes')

{'_id': ObjectId('59fb0d77eee3642f3c0683f7'),
 'energy_KCal': 431,
 'fat_g': '16.6',
 'id': '20-minute-carbonara-with-peas',
 'image': 'https://be35832fa5168a30acd6-5c7e0f2623ae37b4a933167fe83d71b5.ssl.cf3.rackcdn.com/1602/20-minute-carbonara-with-peas__square.jpg',
 'list_ingredients': ['200g spaghetti',
  '250g frozen peas',
  '2 medium eggs',
  '25g Parmesan(grated)',
  '4 level tbsp reduced-fat crème fraîche',
  '1 tbsp olive oil',
  '4 rashers Asda Extra Special Unsmoked Back Bacon(chopped)',
  '1 shallot(finely chopped)',
  '1 garlic clove(crushed)',
  'Chopped parsley(to serve)'],
 'name': '20-minute carbonara with peas',
 'ningredients': 10.0,
 'price_p': '115',
 'salt_mg': '1805',
 'saturated_fat_g': '6.1',
 'serves': '4',
 'sugar_g': '5',
 'time_min': '20',
 'url': 'https://www.asdagoodliving.co.uk/food/recipes/20-minute-carbonara-with-peas'}

In [50]:
rec.server.findOne('ratings')

{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'),
 'rating': 1,
 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'),
 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')}

In [54]:
rec.server.searchInCollection('recipes','_id', rec.server.findOne('ratings')['recipe_id'])

[{'_id': ObjectId('5a073c312cfaa84d14994d8e'),
  'id': 212491,
  'image': 'http://www.seriouseats.com/recipes/assets_c/2012/06/20120627-212491-dinner-tonight-susan-spicer-pan-roasted-chicken-breast-primary-thumb-625xauto-253023.jpg',
  'list_ingredients': ['4 boneless chicken breasts (about 6 ounces each)(with skin still attached)',
   'Salt and pepper',
   '2 tablespoons canola oil or olive oil ',
   '1 cup chicken stock ',
   '6 tablespoons apple cider vinegar ',
   '2 tablespoons butter(softened)',
   '2 teaspoons Dijon mustard',
   '2 teaspoons coarsely chopped tarragon leaves',
   '2 teaspoons chopped fresh chives'],
  'name': "Susan Spicer's Pan-Roasted Chicken Breast with Vinegar, Mustard, and Tarragon",
  'ningredients': 9.0,
  'serves': 'serves 4',
  'time_min': '30 minutes',
  'url': 'http://www.seriouseats.com/recipes/2012/07/susan-spicers-pan-roasted-chicken-breast-with-vinegar-mustard-and-tarragon.html'}]

In [61]:
rec.server.searchInCollection('recipes_ingredients', 'chicken-pumpkin-curry', 'oil')

[{'_id': ObjectId('5a29a690166d3310bc82f132'),
  'chicken-pumpkin-curry': ['ghee',
   'oil',
   'onion',
   'pack',
   'asda',
   'chicken',
   'thigh',
   'fillet',
   'skin',
   'garlic',
   'clove',
   'ginger',
   'turmeric',
   'cumin',
   'asda',
   'garam',
   'masala',
   'chillie',
   'can',
   'coconut',
   'milk',
   'pumpkin',
   'skin',
   'fibre',
   'cornflour',
   'cold',
   'water',
   'make',
   'paste',
   'coriander',
   'rice',
   'serve']}]